In [5]:
import pandas as pd
import requests
from constants import * 

In [6]:
df_weather = pd.read_csv('weather_dublin_hourly.csv')
df_bikes = pd.read_csv('data/dublinbike-historical-data-2021-10.csv')

/tmp/ipykernel_213019/3649067207.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_weather = pd.read_csv('weather_dublin_hourly.csv')


In [7]:
df_weather

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,...,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
0,25-dec-2017 02:00,2,0.0,0,10.9,0,10.0,9.0,11.6,89,...,2,17,2,210,60,62,0.0,14000,14,7
1,25-dec-2017 03:00,0,0.1,0,10.8,0,9.9,8.9,11.5,89,...,2,16,2,210,60,62,0.0,14000,15,7
2,25-dec-2017 04:00,0,1.8,0,9.6,0,9.4,9.2,11.6,97,...,2,13,2,210,61,66,0.0,5000,10,7
3,25-dec-2017 05:00,0,0.9,0,9.0,0,8.4,7.8,10.5,92,...,2,11,2,250,61,66,0.0,10000,10,7
4,25-dec-2017 06:00,0,0.6,0,8.6,0,8.3,8.0,10.7,96,...,2,9,2,220,61,62,0.0,5000,10,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53490,31-jan-2024 20:00,0,0.0,0,6.0,0,5.2,4.1,8.2,88,...,2,9,2,270,2,11,0.0,30000,35,6
53491,31-jan-2024 21:00,0,0.0,0,4.8,0,4.1,3.0,7.6,89,...,2,11,2,270,2,11,0.0,20000,35,6
53492,31-jan-2024 22:00,0,0.0,0,4.6,0,3.7,2.4,7.2,85,...,2,8,2,260,2,11,0.0,20000,35,7
53493,31-jan-2024 23:00,0,0.0,0,4.3,0,3.4,1.9,7.1,85,...,2,7,2,250,2,11,0.0,20000,35,6


### TODO 
# standardize dates to combine weather with historical data
take care with what we use to train (respect to what we can get from the weathe API calls)

In [8]:
df_weather.date = df_weather.date.astype('datetime64[s]')
df_weather.date = pd.to_datetime(df_weather.date).dt.floor('h')
df_weather.date

0       2017-12-25 02:00:00
1       2017-12-25 03:00:00
2       2017-12-25 04:00:00
3       2017-12-25 05:00:00
4       2017-12-25 06:00:00
                ...        
53490   2024-01-31 20:00:00
53491   2024-01-31 21:00:00
53492   2024-01-31 22:00:00
53493   2024-01-31 23:00:00
53494   2024-02-01 00:00:00
Name: date, Length: 53495, dtype: datetime64[s]

In [9]:
df_bikes.TIME = df_bikes.TIME.astype('datetime64[s]')
df_bikes.TIME = pd.to_datetime(df_bikes.TIME).dt.floor('h')
df_bikes.TIME

0        2021-10-01 00:00:00
1        2021-10-01 00:00:00
2        2021-10-01 00:00:00
3        2021-10-01 00:00:00
4        2021-10-01 00:00:00
                 ...        
163379   2021-10-31 23:00:00
163380   2021-10-31 23:00:00
163381   2021-10-31 23:00:00
163382   2021-10-31 23:00:00
163383   2021-10-31 23:00:00
Name: TIME, Length: 163384, dtype: datetime64[s]

In [10]:
df_bikes

,STATION ID,TIME,LAST UPDATED,NAME,BIKE_STANDS,AVAILABLE_BIKE_STANDS,AVAILABLE_BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE
0,2,2021-10-01 00:00:00,2021-09-30 23:56:31,BLESSINGTON STREET,20,7,13,OPEN,Blessington Street,53.3568,-6.26814
1,3,2021-10-01 00:00:00,2021-09-30 23:53:57,BOLTON STREET,20,1,19,OPEN,Bolton Street,53.3512,-6.26986
2,4,2021-10-01 00:00:00,2021-09-30 23:56:37,GREEK STREET,20,11,9,OPEN,Greek Street,53.3469,-6.27298
3,5,2021-10-01 00:00:00,2021-09-30 23:54:50,CHARLEMONT PLACE,40,0,40,OPEN,Charlemont Street,53.3307,-6.26018
4,6,2021-10-01 00:00:00,2021-09-30 23:56:04,CHRISTCHURCH PLACE,20,4,16,OPEN,Christchurch Place,53.3434,-6.27012
...,...,...,...,...,...,...,...,...,...,...,...
163379,113,2021-10-31 23:00:00,2021-10-31 23:29:45,MERRION SQUARE SOUTH,40,32,8,OPEN,Merrion Square South,53.3386,-6.24861
163380,114,2021-10-31 23:00:00,2021-10-31 23:28:04,WILTON TERRACE (PARK),40,30,10,OPEN,Wilton Terrace (Park),53.3337,-6.24834
163381,115,2021-10-31 23:00:00,2021-10-31 23:20:42,KILLARNEY STREET,30,8,22,OPEN,Killarney Street,53.3548,-6.24758
163382,116,2021-10-31 23:00:00,2021-10-31 23:21:27,BROADSTONE,30,18,12,OPEN,Broadstone,53.3547,-6.27231


In [11]:
df_weather

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,...,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
0,2017-12-25 02:00:00,2,0.0,0,10.9,0,10.0,9.0,11.6,89,...,2,17,2,210,60,62,0.0,14000,14,7
1,2017-12-25 03:00:00,0,0.1,0,10.8,0,9.9,8.9,11.5,89,...,2,16,2,210,60,62,0.0,14000,15,7
2,2017-12-25 04:00:00,0,1.8,0,9.6,0,9.4,9.2,11.6,97,...,2,13,2,210,61,66,0.0,5000,10,7
3,2017-12-25 05:00:00,0,0.9,0,9.0,0,8.4,7.8,10.5,92,...,2,11,2,250,61,66,0.0,10000,10,7
4,2017-12-25 06:00:00,0,0.6,0,8.6,0,8.3,8.0,10.7,96,...,2,9,2,220,61,62,0.0,5000,10,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53490,2024-01-31 20:00:00,0,0.0,0,6.0,0,5.2,4.1,8.2,88,...,2,9,2,270,2,11,0.0,30000,35,6
53491,2024-01-31 21:00:00,0,0.0,0,4.8,0,4.1,3.0,7.6,89,...,2,11,2,270,2,11,0.0,20000,35,6
53492,2024-01-31 22:00:00,0,0.0,0,4.6,0,3.7,2.4,7.2,85,...,2,8,2,260,2,11,0.0,20000,35,7
53493,2024-01-31 23:00:00,0,0.0,0,4.3,0,3.4,1.9,7.1,85,...,2,7,2,250,2,11,0.0,20000,35,6


In [12]:
merged_df = pd.merge(df_bikes, df_weather, left_on='TIME', right_on='date', how='left')

In [13]:
merged_df

,STATION ID,TIME,LAST UPDATED,NAME,BIKE_STANDS,AVAILABLE_BIKE_STANDS,AVAILABLE_BIKES,STATUS,ADDRESS,LATITUDE,...,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
0,2,2021-10-01 00:00:00,2021-09-30 23:56:31,BLESSINGTON STREET,20,7,13,OPEN,Blessington Street,53.3568,...,2,9,2,210,2,62,0.0,20000,110,6
1,3,2021-10-01 00:00:00,2021-09-30 23:53:57,BOLTON STREET,20,1,19,OPEN,Bolton Street,53.3512,...,2,9,2,210,2,62,0.0,20000,110,6
2,4,2021-10-01 00:00:00,2021-09-30 23:56:37,GREEK STREET,20,11,9,OPEN,Greek Street,53.3469,...,2,9,2,210,2,62,0.0,20000,110,6
3,5,2021-10-01 00:00:00,2021-09-30 23:54:50,CHARLEMONT PLACE,40,0,40,OPEN,Charlemont Street,53.3307,...,2,9,2,210,2,62,0.0,20000,110,6
4,6,2021-10-01 00:00:00,2021-09-30 23:56:04,CHRISTCHURCH PLACE,20,4,16,OPEN,Christchurch Place,53.3434,...,2,9,2,210,2,62,0.0,20000,110,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163379,113,2021-10-31 23:00:00,2021-10-31 23:29:45,MERRION SQUARE SOUTH,40,32,8,OPEN,Merrion Square South,53.3386,...,2,17,2,260,80,82,0.0,20000,15,7
163380,114,2021-10-31 23:00:00,2021-10-31 23:28:04,WILTON TERRACE (PARK),40,30,10,OPEN,Wilton Terrace (Park),53.3337,...,2,17,2,260,80,82,0.0,20000,15,7
163381,115,2021-10-31 23:00:00,2021-10-31 23:20:42,KILLARNEY STREET,30,8,22,OPEN,Killarney Street,53.3548,...,2,17,2,260,80,82,0.0,20000,15,7
163382,116,2021-10-31 23:00:00,2021-10-31 23:21:27,BROADSTONE,30,18,12,OPEN,Broadstone,53.3547,...,2,17,2,260,80,82,0.0,20000,15,7


In [14]:
# Specify the timestamp you want to filter for
timestamp_to_find = '2021-10-31 23:00:00'

# Using .loc[]
result = merged_df.loc[merged_df['TIME'] == timestamp_to_find]

print(result)


        STATION ID                TIME         LAST UPDATED  \
163164           2 2021-10-31 23:00:00  2021-10-31 22:57:42   
163165           3 2021-10-31 23:00:00  2021-10-31 22:55:41   
163166           4 2021-10-31 23:00:00  2021-10-31 22:54:01   
163167           5 2021-10-31 23:00:00  2021-10-31 22:59:38   
163168           6 2021-10-31 23:00:00  2021-10-31 22:56:13   
...            ...                 ...                  ...   
163379         113 2021-10-31 23:00:00  2021-10-31 23:29:45   
163380         114 2021-10-31 23:00:00  2021-10-31 23:28:04   
163381         115 2021-10-31 23:00:00  2021-10-31 23:20:42   
163382         116 2021-10-31 23:00:00  2021-10-31 23:21:27   
163383         117 2021-10-31 23:00:00  2021-10-31 23:23:39   

                         NAME  BIKE_STANDS  AVAILABLE_BIKE_STANDS  \
163164     BLESSINGTON STREET           20                     10   
163165          BOLTON STREET           20                      8   
163166           GREEK STREET       

In [15]:
result.iloc[0]

STATION ID                                 2
TIME                     2021-10-31 23:00:00
LAST UPDATED             2021-10-31 22:57:42
NAME                      BLESSINGTON STREET
BIKE_STANDS                               20
AVAILABLE_BIKE_STANDS                     10
AVAILABLE_BIKES                           10
STATUS                                  OPEN
ADDRESS                   Blessington Street
LATITUDE                             53.3568
LONGITUDE                           -6.26814
date                     2021-10-31 23:00:00
ind                                        2
rain                                     0.0
ind.1                                      0
temp                                     8.4
ind.2                                      0
wetb                                     7.6
dewpt                                    6.6
vappr                                    9.8
rhum                                      89
msl                                    982.8
ind.3     

In [16]:
# Specify the timestamp you want to filter for
timestamp_to_find = '2021-10-31 23:00:00'

# Using .loc[]
result = df_weather.loc[df_weather.date == timestamp_to_find]

print(result)

                     date  ind  rain  ind.1  temp  ind.2  wetb  dewpt  vappr  \
33765 2021-10-31 23:00:00    2   0.0      0   8.4      0   7.6    6.6    9.8   

       rhum  ...  ind.3  wdsp  ind.4  wddir  ww   w  sun    vis clht  clamt  
33765    89  ...      2    17      2    260  80  82  0.0  20000   15      7  

[1 rows x 21 columns]


Perfect, the merge is looking up and finding the rate weather info in the dataset